In [2]:
from osmhandlers import GetPublicTransportMembersHandler, NetworkBuilder
#from gtfsprocessor import GTFSHandler

In [3]:
OSM_FILE = 'porto.osm'
#GTFS_FILE = 'gtfs_mdp_11_09_2023.zip'
GTFS_FILE = 'gtfs-stcp-2023-09.zip'  
OUTPUT_NETWORK_FILE = 'network.xml'
OUTPUT_TRANSIT_SCHEDULE_FILE = 'transitSchedule.xml'
OUTPUT_TRANSIT_VEHICLES_FILE = 'transitVehicles.xml'

In [4]:
publicTransportMembersHandler = GetPublicTransportMembersHandler()
publicTransportMembersHandler.apply_file(OSM_FILE)

In [5]:
networkBuilder = NetworkBuilder(publicTransportMembersHandler.publicTransportMembers)
networkBuilder.apply_file(OSM_FILE)

In [16]:
from helper import *
import gtfs_kit as gk
from xml.dom import minidom 
from haversine import haversine
import numpy as np
from pyreproj import Reprojector

class GTFSHandler:
    def __init__(self, feedFileName, links, relations, wayLinks, units='km'):
        self.feed = gk.read_feed(feedFileName, dist_units=units)
        self.links = links
        self.realtions = relations
        self.wayLinks = wayLinks
        self.stop_links = {}
        self.vehicles = []
        self.stopFacilities = set()
        reprojector = Reprojector()
        self.coordinateTransformer = reprojector.get_transformation_function(from_srs="WGS84", to_srs="EPSG:3857")

    def __str__(self):
        return self.xmlRoot.toprettyxml(indent="\t")
    
    def createRouteRelations(self):
        self.agency_relations = {}
        for rID, relation in self.realtions.items():
            tags = relation['tags']
            if  get(tags, 'operator') == self.agency_name\
             or get(tags, 'operator:wikipedia') == self.agency_name\
             or get(tags, 'operator') == f"pt:{self.agency_name}"\
             or get(tags, 'operator:wikipedia') == f"pt:{self.agency_name}"\
             or get(tags, 'network') == self.agency_name\
             or get(tags, 'operator') == self.agency_id\
             or get(tags, 'operator:wikipedia') == self.agency_id\
             or get(tags, 'operator') == f"pt:{self.agency_id}"\
             or get(tags, 'operator:wikipedia') == f"pt:{self.agency_id}"\
             or get(tags, 'network') == self.agency_id:
                pathMembers = [id for id in relation['members'] if id in self.wayLinks]
                route_f = []
                route_r = []
                size = len(pathMembers)
                for i in range(size):
                    route_f += self.wayLinks[pathMembers[i]]['f']
                    route_r += self.wayLinks[pathMembers[size-i-1]]['r'][::-1]
                self.agency_relations[get(tags, 'ref')] = {'f':route_f,'r':route_r}
    
    def createProfiles(self):
        profiles = {r['route_id']:{} for _,r in self.feed.routes.iterrows()}

        for trip_id in self.stop_times['trip_id'].unique():
            trip = self.stop_times[self.stop_times['trip_id'] == trip_id].sort_values(by='stop_sequence')
            profile = [(trip.iloc[0]['stop_id'], '00:00:00', (trip.iloc[0]["stop_lat"], trip.iloc[0]["stop_lon"]), trip.iloc[0]["stop_name"])]
            routeId = trip.iloc[0]['route_id']
            
            for i in range(1,len(trip)):
                diff = (parse_time(trip.iloc[i]['arrival_time']) - parse_time(trip.iloc[i-1]['arrival_time'])) + parse_time(profile[i-1][1])
                profile.append((trip.iloc[i]['stop_id'], str(diff), (trip.iloc[i]["stop_lat"], trip.iloc[i]["stop_lon"]), trip.iloc[i]["stop_name"] ))
            
            profileRouteID = '>'.join([f'{s}' for s,_,_,_ in profile])
            profileID = '>'.join([f'{s}::{t}' for s, t,_,_ in profile])
            
            if profileRouteID in profiles[routeId]:
                if profileID in profiles[routeId][profileRouteID]:
                    profiles[routeId][profileRouteID][profileID]['trips'].append(trip_id)
                else:
                    profiles[routeId][profileRouteID][profileID] = {'profile':profile,'trips':[trip_id]}
            else:
                profiles[routeId][profileRouteID] = {profileID: {'profile':profile,'trips':[trip_id]} }

        self.profiles = profiles
    
    def setupXML(self):
        imp = minidom.getDOMImplementation()
        doctype = imp.createDocumentType("transitSchedule", None, "http://www.matsim.org/files/dtd/transitSchedule_v2.dtd")
        root = imp.createDocument(None, "transitSchedule", doctype)
        self.xmlRoot = root

        attributes = self.xmlRoot.createElement("attributes")

        #CHECK THIS DATES IN THE FUTURE
        createElement(self.xmlRoot, "attribute", '2024-01-08', attributes, name="endDate", __class="java.lang.String")
        createElement(self.xmlRoot, "attribute", '2024-01-08', attributes, name="startDate", __class="java.lang.String")

        self.xmlRoot.documentElement.appendChild(attributes)

        self.transitStops = createElement(self.xmlRoot, "transitStops", __parent__= self.xmlRoot.documentElement)

    def inverseLinkId(self, id):
        if '_r' in id: return id.replace('_r', '')
        return f'{id}_r'
    
    def correctRoutes(self, route):

        if route == None:
            return None

        #Does first link connect to anything?
        for linkId in route[1:]:
            if self.links[route[0]]['to'] == self.links[linkId]['to'] or self.links[route[0]]['to'] == self.links[linkId]['from']:
                break
        else:
            if self.inverseLinkId(route[0]) in self.links:
                route[0] = self.inverseLinkId(route[0])

        firstLinkId = route[0]
        lastLinkId = route[-1]

        newRoute = [firstLinkId]
 
        #print(newRoute[0])
        currLinkId = firstLinkId
        while currLinkId != lastLinkId:
            #print(f"----{currLinkId}-{self.links[currLinkId]['to']}")
            for linkId in route:
                if linkId in newRoute or self.inverseLinkId(linkId) in newRoute:
                    continue

                #print(f"----------{linkId}-{self.links[linkId]['from']}")
                
                if self.links[currLinkId]['to'] == self.links[linkId]['from']:
                    newRoute.append(linkId)
                    currLinkId = linkId
                    #print(linkId)
                    break
                
                inverseLinkId = self.inverseLinkId(linkId)

                #print(f"----------{inverseLinkId}-{self.links[inverseLinkId]['from']}")

                if inverseLinkId in self.links:
                    if self.links[currLinkId]['to'] == self.links[inverseLinkId]['from']:
                        newRoute.append(inverseLinkId)
                        currLinkId = inverseLinkId
                        #print(inverseLinkId)
                        break
            else:
                return None

        return newRoute

    def findRoute(self, route_profile, route_name):
        firstStopIndex = -1
        route = None
        direction = None
        for i, (stop, _, location, _) in enumerate(list(route_profile.values())[0]['profile']):
            for j in range(len(self.agency_relations[route_name]['f'])):
                linkFront = self.links[self.agency_relations[route_name]['f'][i]]
                distFront = min(haversine(location, linkFront['from']), haversine(location, linkFront['to']))
                if distFront <= 0.5:
                    route = self.agency_relations[route_name]['f']
                    direction = 'f'
                    break
            
            if route != None:
                firstStopIndex = i
                break
            
            for j, linkId in enumerate(self.agency_relations[route_name]['r']):
                dist = min(haversine(location, self.links[linkId]['from']), haversine(location, self.links[linkId]['to']))
                if dist < 0.5:
                    route = self.agency_relations[route_name]['r']
                    direction = 'r'
                    break
            if route != None:
                firstStopIndex = i
                break
        
        return route, firstStopIndex, direction
    
    def createStopFacilitiesInRoute(self, firstStopIndex, profile, route, routeProfile, route_name, direction):
        last_link = 0
        for i in range(firstStopIndex, len(profile)):
            stop, time, location, name = profile[i]

            stopId = f"{stop}_{route_name}_{direction}"
            createElement(self.xmlRoot, 'stop', __parent__=routeProfile, refId=stopId, arrivalOffset=time, departureOffset=time, awaitDeparture='true')

            if stopId in self.stopFacilities:
                continue

            dist = 999999
            closest = None
            for j in range(last_link, len(route)):
                link = self.links[route[j]]

                tmpDist = min(haversine(location, link['from']), haversine(location, link['to']))
                if tmpDist < dist:
                    dist = tmpDist
                    closest = route[j]
                    last_link = j

            x,y = self.coordinateTransformer(location[0], location[1])
            createElement(self.xmlRoot, 'stopFacility', __parent__=self.transitStops, id=stopId, x=x, y=y, linkRefId=closest ,name=name, isBlocking='false')

            self.stopFacilities.add(stopId)

    def createTransitRoutes(self, route_id, route_profile, transitLine, subrouteId):
        route_name = self.feed.routes[self.feed.routes['route_id'] == route_id].iloc[0]['route_short_name']

        route, firstStopIndex, direction = self.findRoute(route_profile, route_name)

        route = self.correctRoutes(route)

        if route == None:
            print(f"Inconsistencies in route {route_name}, subroute {subrouteId}, skipping it...")
            return
        print(route)

        for i, (_, profileDesc) in enumerate(route_profile.items()):
            profile = profileDesc['profile']
            departureTrips = profileDesc['trips']
            
            transitRoute = createElement(self.xmlRoot, "transitRoute", __parent__=transitLine, id=f"R-{self.agency_id}-{route_id}-{i}")
            createElement(self.xmlRoot, "transportMode", self.transportMode, transitRoute)
            routeProfile = createElement(self.xmlRoot, "routeProfile", __parent__=transitRoute)
            routeXml = createElement(self.xmlRoot, 'route', __parent__=transitRoute)
            departures = createElement(self.xmlRoot, "departures", __parent__=transitRoute)
            
            self.createStopFacilitiesInRoute(firstStopIndex, profile, route, routeProfile, route_name, direction)

            for trip in departureTrips:
                departureTime=self.stop_times[self.stop_times['trip_id'] == trip]['arrival_time'].min()
                createElement(self.xmlRoot, 'departure', __parent__=departures, id=trip, departureTime=fix_time(departureTime), vehicleRefId=f'{self.agency_id}_{trip}')
                self.vehicles.append(f'{self.agency_id}_{trip}')
            
            
            for linkId in route:
                createElement(self.xmlRoot, 'link', __parent__=routeXml, refId=linkId)
                    

    def createTransitLines(self):
        for route_id, route_route in self.profiles.items():
            route_type = self.feed.routes[self.feed.routes["route_id"] == route_id].iloc[0]["route_type"]
            
            for i, (_, route_profile) in enumerate(route_route.items()):
                try:
                    transitLine = createElement(self.xmlRoot, "transitLine",  __parent__=self.xmlRoot.documentElement, id=f"L-{self.agency_id}-{route_id}-{i}") 
                    attributes = createElement(self.xmlRoot, "attributes", __parent__=transitLine)
                    createElement(self.xmlRoot, "attribute", self.agency_id, attributes, name="gtfs_agency_id", __class="java.lang.String")
                    createElement(self.xmlRoot, "attribute", route_id, attributes, name="gtfs_route_short_name", __class="java.lang.String")
                    createElement(self.xmlRoot, "attribute", str(route_type), attributes, name="gtfs_route_type", __class="java.lang.String")

                    self.createTransitRoutes(route_id, route_profile, transitLine, i)
                except:
                    print(f"EXCEPT Inconsistencies in route {route_id}, subroute {i}, skipping it...")
                break
            break
            

    def processGTFS(self):
        self.agency_name = self.feed.agency.iloc[0]['agency_name']
        self.agency_id = self.agency_name.replace(' ','_') if self.feed.agency['agency_id'].isna()[0] else self.feed.agency['agency_id'][0]

        self.transportMode = "light_rail"

        self.stop_times = self.feed.stop_times.merge(self.feed.trips, left_on="trip_id", right_on="trip_id")\
                                         .merge(self.feed.stops, left_on="stop_id", right_on="stop_id")\
                                         .merge(self.feed.calendar, left_on='service_id', right_on='service_id')\
                                         .merge(self.feed.routes, left_on='route_id', right_on='route_id')\
                                         .sort_values(by=['trip_id','route_id', 'arrival_time'])
        
        self.stop_times = self.stop_times[self.stop_times['monday'] == 1]

        print('creating profiles')
        self.createProfiles()
        print('creating relation')
        self.createRouteRelations()
        print('setting up xml')
        self.setupXML()
        print('creating lines')
        self.createTransitLines()

gtfsHandler = GTFSHandler(GTFS_FILE, networkBuilder.ptLinks, publicTransportMembersHandler.relations, networkBuilder.wayLinks)
gtfsHandler.processGTFS()

creating profiles
creating relation
setting up xml
creating lines
['11877167780', '7389947280', '7389947281', '383366430', '383366431', '11877167770', '11877167771', '4348784013_r', '4348784012_r', '4348784011_r', '4348784010_r', '11746231551_r', '11746231550_r', '4348799261_r', '4348799260_r', '2395938952_r', '2395938951_r', '2395938950_r', '11746231530_r', '11746231520_r', '2202328933_r', '2202328932_r', '2202328931_r', '2202328930_r', '1901284870', '1901284871', '1901284872', '1901284873', '1901284874', '1901284875', '2151622430', '2151622431', '261199000', '261199001', '261199002', '261199003', '10011663290', '11745417220', '2152003490', '2152003491', '1245398280', '1245398281', '1245398282', '1245398283', '1245398284', '1245398285', '1245398286', '1245398287', '1901284900', '1901284901', '1901284902', '261198440', '261198441', '261198442', '261198443', '261198444', '261198445', '261198446', '261198447', '2169698090', '2169698091', '2169698092', '2169698093', '2169698094', '2169698

In [20]:
def getVehiclesXML(gtfsHandler):
   xmlRoot = minidom.Document()
   rootElement = createElement(xmlRoot, "vehicleDefinitions", __parent__=xmlRoot, xmlns="http://www.matsim.org/files/dtd")
   rootElement.setAttribute("xmlns:xsi","http://www.w3.org/2001/XMLSchema-instance")
   rootElement.setAttribute("xsi:schemaLocation","http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v2.0.xsd")

   seats = 101
   standing = 0
   lenght = 7.5
   width = 1.0
   pce = 1.0
   flowEfficiencyFactor = 1.0

   vehicleTypeId = f'{gtfsHandler.agency_id}_vehicle'
   vehicleType = createElement(xmlRoot, 'vehicleType', __parent__=rootElement, id=vehicleTypeId)

   createElement(xmlRoot, 'capacity', __parent__=vehicleType, seats=seats, standingRoomInPersons=standing)
   createElement(xmlRoot, 'length', __parent__=vehicleType, meter=lenght)
   createElement(xmlRoot, 'width', __parent__=vehicleType, meter=width)
   createElement(xmlRoot, 'costInformation', __parent__=vehicleType)
   createElement(xmlRoot, 'passengerCarEquivalents', __parent__=vehicleType, pce=pce)
   createElement(xmlRoot, 'networkMode', __parent__=vehicleType, networkMode=gtfsHandler.transportMode)
   createElement(xmlRoot, 'flowEfficiencyFactor', __parent__=vehicleType, factor=flowEfficiencyFactor)

   for vehicleId in gtfsHandler.vehicles:
      createElement(xmlRoot, 'vehicle', __parent__=xmlRoot.documentElement, id=vehicleId, type=vehicleTypeId)

   return xmlRoot.toprettyxml(indent="\t")

In [18]:
with open(OUTPUT_NETWORK_FILE, 'w') as file:
        file.write(str(networkBuilder))

In [17]:
with open(OUTPUT_TRANSIT_SCHEDULE_FILE, 'w') as file:
        file.write(str(gtfsHandler))

In [21]:
with open(OUTPUT_TRANSIT_VEHICLES_FILE, 'w') as file:
        file.write(str(getVehiclesXML(gtfsHandler)))